# 08_Facility_Overlays

This notebook overlays each type of health facility on the Rural/Urban classification map of LSOAs in the South West.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.geometry import Point

PROCESSED_DIR = os.path.join('data','processed')
FACILITY_DIR = os.path.join(PROCESSED_DIR,'Health Infrastructure','enriched')
LSOA_GPKG = os.path.join(PROCESSED_DIR,'lsoa_sw_enriched_demographics.gpkg')

lsoa_sw_gdf = gpd.read_file(LSOA_GPKG)


In [ ]:
ruc_palette = {
    'Urban: Nearer to a major town or city': '#377eb8',
    'Urban: Further from a major town or city': '#4daf4a',
    'Smaller rural: Nearer to a major town or city': '#984ea3',
    'Smaller rural: Further from a major town or city': '#ff7f00',
    'Larger rural: Nearer to a major town or city': '#e41a1c',
    'Larger rural: Further from a major town or city': '#ffff33',
}
print(lsoa_sw_gdf['RUC21NM'].value_counts())


In [ ]:
facility_files = [
    ('Community Diagnostic Centres','NHS_SW_ Community_Diagnostic_Centres_enriched.csv'),
    ('GP Practices and Polyclinics','NHS_SW_ GP_Practices_and_Polyclinics_enriched.csv'),
    ('Acute Hospitals','NHS_SW_Acute_Hospitals_enriched.csv'),
    ('Ambulance Stations','NHS_SW_Ambulance_Stations_enriched.csv'),
    ('Community Hospitals','NHS_SW_Community_Hospitals_enriched.csv'),
    ('Community and Outreach Facilities','NHS_SW_Community_and_Outreach_Facilities_enriched.csv'),
    ('Specialist Hospitals','NHS_SW_Specialist_Hospitals_enriched.csv'),
]
for label, fname in facility_files:
    df = pd.read_csv(os.path.join(FACILITY_DIR, fname))
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']), crs='EPSG:4326')
    gdf = gdf.to_crs(lsoa_sw_gdf.crs)

    fig, ax = plt.subplots(figsize=(11,8))
    lsoa_sw_gdf.plot(column='RUC21NM', ax=ax,
                     color=lsoa_sw_gdf['RUC21NM'].map(ruc_palette),
                     edgecolor='grey', linewidth=0.2, legend=False)
    gdf.plot(ax=ax, markersize=12, color='black', marker='o', label=label)
    legend_patches = [mpatches.Patch(color=c, label=l) for l,c in ruc_palette.items()]
    handles = legend_patches + [plt.Line2D([0],[0], marker='o', color='black', linestyle='', label=label)]
    ax.legend(handles=handles, title='RUC21NM', loc='lower left', fontsize=8, title_fontsize=9, frameon=True)
    ax.set_title(f'{label} over Rural/Urban Classification')
    ax.axis('off')
    plt.tight_layout()
    plt.show()
